In [ ]:
import numpy as np
from numpy.random import randn

import matplotlib.pyplot as plt
import pandas as pd

from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm

In [ ]:
# Generate synthetic data
np.random.seed(0)
n = 100
a  = np.zeros(n) # create an array of 0's of length n
s = 1 # controls how fast true mean moves
a[0] = 100
for i in range(1,n):
    a[i] = a[i-1] + s*randn()
noise_std = 0.8
x = a + noise_std*randn(n)

plt.scatter(range(n), x, marker='.', color='black')
p = plt.plot(a)

In [ ]:
# fit = SimpleExpSmoothing(x).fit(smoothing_level=0.1,optimized=False) # choose smoothing level explicitly
fit = SimpleExpSmoothing(x).fit() # choose smoothing level to best fit data
xhat = fit.fittedvalues
plt.scatter(range(n), x, marker='.', color='black')
p = plt.plot(a)
plt.plot(range(n), xhat, color='red')

* Change noise std and s and see how it affects x.
* Change the alpha parameter to see how it affects the red line, and to make the red line track x.
* Remove the arguments to fit() to let SimpleExpSmoothing fit alpha for you.
* How does the optimal value of alpha change as you change noise std and s?

In [ ]:
# fit.params shows the chosen values for all parameters 
fit.params

In [ ]:
# forecast, too!
xhat = fit.fittedvalues
l = 10
forecast = fit.forecast(l)

# plot with forecast
plt.scatter(range(n), x, marker='.', color='black')
p = plt.plot(a)
plt.plot(range(n), xhat, color='red')
plt.plot(range(n,n+l), forecast, color='red')

# Holt's method: add a trend

In [ ]:
np.random.seed(0)
n = 100
a = np.zeros(n)
b = np.zeros(n)
a[0] = 100 # initial level
b[0] = 1 # initial trend

for i in range(1,n):
    b[i] = b[i-1] + 0.2*randn()      # trend changes slowly
    a[i] = a[i-1] + b[i-1] + randn() # level moves with the trend + extra random movement

# We observe the level + extra noise
x = a + .9*randn(n) 

# fit the model
fit = Holt(x).fit(smoothing_level=0.2,smoothing_slope=.5,optimized=False)
# fit = Holt(x).fit()
xhat = fit.fittedvalues

# plot
plt.scatter(range(n), x, marker='.', color='black')
p = plt.plot(a)
plt.plot(range(n), xhat, color='red')

Suppose I change the code to set initial value of trend b[0] = -1. What will happen?
* (up) The blue line will trend downward
* (down) The blue line will trend upward more quickly
* (yes) The blue line will stay at 0
* (no) The blue line will bounce around more
* (coffee) The black dots will be further away from the black line.

Suppose I change 0.8 to 5 in defining the observations `x = a + 0.8*randn(n)`. What will happen?
* (up) The blue line will trend downward
* (down) The blue line will trend upward more quickly
* (yes) The blue line will stay at 0
* (no) The blue line will bounce around more
* (coffee) The black dots will be further away from the black line.

Suppose I multiply `randn()` by 3 in defining the level `a[i] = a[i-1] + b[i-1] + randn()`. What will happen?
* (up) The blue line will trend downward
* (down) The blue line will trend upward more quickly
* (yes) The blue line will stay at 0
* (no) The blue line will bounce around more
* (coffee) The black dots will be further away from the black line.

In [ ]:
# fit 
fit = Holt(x).fit()
xhat = fit.fittedvalues

# forecast, too!
xhat = fit.fittedvalues
l = 10 # how far out to forecast
forecast = fit.forecast(l)

# plot with forecast
plt.scatter(range(n), x, marker='.', color='black')
p = plt.plot(a)
plt.plot(range(n), xhat, color='red')
plt.plot(range(n,n+l), forecast, color='red')

# Holt-Winters: Add seasonality, too!

Let's show how to use the `ExponentialSmoothing` class to control how we fit the level, slope, and seasonality.

In [ ]:
ExponentialSmoothing?

In [ ]:
# call method with just a timeseries 
# -> fits a model with just a level (no slope or seasonality) 
# -> learns the smoothing_level parameter
model = ExponentialSmoothing(x).fit()
model.params

In [ ]:
# you can control the smoothing_level parameter yourself if you prefer
model = ExponentialSmoothing(x).fit(smoothing_level=0.2)
model.params

In [ ]:
# can also add an (additive or multiplicative) trend 
model = ExponentialSmoothing(x, trend='add').fit()
model.params

In [ ]:
# you can control the smoothing_level and smoothing_slope parameters yourself if you prefer
model = ExponentialSmoothing(x).fit(smoothing_level=0.2,smoothing_slope=.5) 
model.params

In [ ]:
# can add a trend and seasonality
model = ExponentialSmoothing(x, trend='add', seasonal='mul', seasonal_periods=12).fit()
model.params

In [ ]:
# can control parameters explicitly
# -> can control some parameters and optimize over others
model = ExponentialSmoothing(x, trend='add', seasonal='mul', seasonal_periods=12)
model.fit(smoothing_seasonal=.5)
model.params

# Application: predicting co2 levels

In [ ]:
from sklearn import datasets

In [ ]:
co2 = sm.datasets.co2.load()

In [ ]:
co2 = pd.DataFrame(co2.data)
co2.head()

In [ ]:
dates = [pd.to_datetime(int(d), format = '%Y%m%d') for d in co2['date']]
co2['date'] = dates
co2.head()

# Application: predicting sales

In [ ]:
pd.read_csv("SalesAndForecastsTimeSeries.csv")